In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import yfinance as yf
import datetime
from sklearn.preprocessing import MinMaxScaler

## Import data

In [ ]:
start_date = datetime.datetime(2017, 10, 31)
end_date = datetime.datetime(2024, 1, 1)
btc_info = yf.Ticker("BTC-USD")

# pass the parameters as the taken dates for start and end
df = btc_info.history(start = start_date, end = end_date)

In [ ]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-10-31 00:00:00+00:00,6132.020020,6470.430176,6103.330078,6468.399902,2311379968,0.0,0.0
2017-11-01 00:00:00+00:00,6440.970215,6767.310059,6377.879883,6767.310059,2870320128,0.0,0.0
2017-11-02 00:00:00+00:00,6777.770020,7367.330078,6758.720215,7078.500000,4653770240,0.0,0.0
2017-11-03 00:00:00+00:00,7087.529785,7461.290039,7002.939941,7207.759766,3369860096,0.0,0.0
2017-11-04 00:00:00+00:00,7164.479980,7492.859863,7031.279785,7379.950195,2483800064,0.0,0.0


In [ ]:
df = df.drop(columns=['Dividends', 'Stock Splits'])

In [ ]:
df.columns = ['open', 'high', 'low', 'close', 'vol']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2253 entries, 2017-10-31 00:00:00+00:00 to 2023-12-31 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    2253 non-null   float64
 1   high    2253 non-null   float64
 2   low     2253 non-null   float64
 3   close   2253 non-null   float64
 4   vol     2253 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 105.6 KB


### Feature add

In [ ]:
# Moving Average function
def moving_average(data, period):
    return data.rolling(window=period).mean()

# Calculate RSI function
def calculate_rsi(data, period=14):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [ ]:
# Adding new columns for 10-day and 30-day moving averages
df['MA10'] = moving_average(df['close'], 10)
df['MA30'] = moving_average(df['close'], 30)

# Adding a new column for RSI
df['RSI'] = calculate_rsi(df['close'], 14)


In [ ]:
df = df[df.index > '2018-01-01']

In [ ]:
# Prepare the volume and price differences, normalize volume
BTC_vol = df["vol"].values
df_diff = df.diff().dropna()
df_diff["vol"] = np.log(1 + BTC_vol[:-1])

In [ ]:
df_aligned = df.loc[df_diff.index]

### Train, Test split

In [ ]:
# Train data
# Period : From start of 2018 to end of 2022
mask_train = (df_diff.index >= "2018-01-01") & (df_diff.index < "2023-01-01")
df_train = df_diff.loc[mask_train].copy()
train_close = df_aligned.loc[mask_train, "close"].values
df_train["Relative_Close"] = train_close / train_close[0]

In [ ]:
# Test data
# Period : Whole 2023
mask_test = (df_diff.index >= "2023-01-01") & (df_diff.index < "2024-01-01")  # December 2018 for testing
df_test = df_diff.loc[mask_test].copy()
test_close = df_aligned.loc[mask_test, "close"].values
df_test["Relative_Close"] = test_close / train_close[0]


In [ ]:
df_train.head()

,open,high,low,close,vol,MA10,MA30,RSI,Relative_Close
Date,,,,,,,,,
2018-01-03 00:00:00+00:00,1353.200195,128.200195,1680.900391,218.900391,23.547415,127.520020,118.126660,4.522984,1.000000
2018-01-04 00:00:00+00:00,292.500000,166.900391,-322.299805,398.200195,23.548915,157.260059,122.750000,4.629498,1.026196
2018-01-05 00:00:00+00:00,206.500000,1965.499023,680.599609,1830.299805,23.804405,132.970020,104.600000,15.151098,1.146602
2018-01-06 00:00:00+00:00,1984.899414,7.201172,1561.799805,97.500000,23.894668,168.850000,-12.423307,-2.332564,1.153016
2018-01-07 00:00:00+00:00,65.201172,-132.800781,-676.899414,-1049.400391,23.630964,187.109961,-3.060026,-1.421186,1.083981


In [ ]:
df_test.head()

,open,high,low,close,vol,MA10,MA30,RSI,Relative_Close
Date,,,,,,,,,
2023-01-01 00:00:00+00:00,-55.759766,1.453125,3.714844,77.583984,23.142672,-20.526172,-15.452669,3.587225,1.093683
2023-01-02 00:00:00+00:00,77.595703,128.904297,50.994141,63.390625,22.947280,-10.848242,-7.325521,12.652816,1.097853
2023-01-03 00:00:00+00:00,63.337891,1.103516,50.142578,-8.613281,23.216287,-16.789844,-15.020964,-20.369304,1.097287
2023-01-04 00:00:00+00:00,-8.642578,204.138672,45.392578,183.380859,23.355376,2.125195,-3.719596,13.807905,1.109351
2023-01-05 00:00:00+00:00,183.267578,-80.564453,122.519531,-26.501953,23.636798,-8.306836,-8.425586,-1.848586,1.107607


In [ ]:
# Generate dataset function
def generate_dataset(df, seq_len):
    X_list, y_list = [], []
    for i in range(len(df) - seq_len):
        X_list.append(df.iloc[i:(i+seq_len), :].values)
        y_list.append(df["close"].iloc[i + seq_len])
    return np.array(X_list), np.array(y_list)

In [ ]:
LAG = 1

In [ ]:
# # Prepare training and test datasets
# X_train, y_train = generate_dataset(df_train, LAG)
# X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [ ]:
validation_size = 0.2
n_validation = int(len(df_train) * validation_size)

df_val = df_train.iloc[-n_validation:]
df_train_reduced = df_train.iloc[:-n_validation]

X_train, y_train = generate_dataset(df_train_reduced, LAG)
X_val, y_val = generate_dataset(pd.concat((df_train_reduced.iloc[-LAG:], df_val)), LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [ ]:
num_samples, num_timesteps, num_features = X_train.shape
X_train_reshaped = X_train.reshape(-1, num_features)
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_train_scaled = X_train_scaled.reshape(num_samples, num_timesteps, num_features)

In [ ]:
num_samples_val, num_timesteps, num_features = X_val.shape
X_val_reshaped = X_val.reshape(-1, num_features)
X_val_scaled = scaler.transform(X_val_reshaped)
X_val_scaled = X_val_scaled.reshape(num_samples_val, num_timesteps, num_features)

In [ ]:
num_samples_test, num_timesteps, num_features = X_test.shape
X_test_reshaped = X_test.reshape(-1, num_features)
X_test_scaled = scaler.transform(X_test_reshaped)
X_test_scaled = X_test_scaled.reshape(num_samples_test, num_timesteps, num_features)

### Hyperparameter Tuning

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

<ipython-input-24-bf4685511596>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
def build_model(hp):
    model = Sequential([
        LSTM(
            units=hp.Int('units1', min_value=32, max_value=256, step=32),
            return_sequences=True,
            input_shape=(X_train.shape[1], X_train.shape[2]),
            recurrent_dropout=hp.Float('recurrent_dropout1', min_value=0.0, max_value=0.5, step=0.1)
        ),
        Dropout(rate=hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)),
        LSTM(
            units=hp.Int('units2', min_value=32, max_value=256, step=32),
            recurrent_dropout=hp.Float('recurrent_dropout2', min_value=0.0, max_value=0.5, step=0.1)
        ),
        Dropout(rate=hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)),
        Dense(
            units=hp.Int('dense_units', min_value=16, max_value=128, step=16),
            activation='relu'
        ),
        Dense(1)
    ])

    model.compile(
        optimizer=Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='mse'
    )

    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,  # Number of different configurations to try
    executions_per_trial=1,  # Number of models to train for each trial
    directory='my_dir',  # Directory to save logs and models
    project_name='lstm_tuning'
)

In [ ]:
# Display search space summary
tuner.search_space_summary()

Search space summary
Default search space size: 8
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
recurrent_dropout1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dropout1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
recurrent_dropout2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dropout2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001

In [ ]:
# Perform the hyperparameter search
tuner.search(
    X_train_scaled, y_train,
    epochs=5,
    validation_data=(X_val_scaled, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
)

Trial 10 Complete [00h 00m 09s]
val_loss: 1026283.75

Best val_loss So Far: 1026282.875
Total elapsed time: 00h 01m 50s


In [ ]:
# Initialize an empty list to hold each trial's data
trial_data = []

# Iterate through each trial and collect the data
for trial_id, trial in tuner.oracle.trials.items():
    if trial.status == "COMPLETED":
        # Extract the final validation loss for the trial
        val_loss = trial.metrics.get_best_value('val_loss')
        # Prepare a dictionary for the trial
        trial_info = {
            'Trial ID': trial_id,
            'MSE': val_loss
        }
        # Update the dictionary with the hyperparameters
        trial_info.update(trial.hyperparameters.values)
        # Append the dictionary to the list
        trial_data.append(trial_info)

# Convert the list of dictionaries to a DataFrame
df_trials = pd.DataFrame(trial_data)

# Display the DataFrame
df_trials

,Trial ID,MSE,units1,recurrent_dropout1,dropout1,units2,recurrent_dropout2,dropout2,dense_units,learning_rate
0,00,1.026283e+06,256,0.0,0.1,256,0.3,0.2,96,0.000158
1,01,1.026295e+06,64,0.4,0.4,96,0.0,0.1,48,0.001355
2,02,1.026288e+06,32,0.3,0.1,256,0.4,0.1,96,0.004863
3,03,1.026283e+06,160,0.3,0.2,160,0.4,0.0,112,0.000103
4,04,1.026285e+06,256,0.4,0.0,224,0.0,0.4,64,0.000222
5,05,1.026297e+06,96,0.0,0.2,128,0.4,0.3,128,0.002500
6,06,1.026289e+06,128,0.1,0.4,224,0.0,0.4,96,0.000386
7,07,1.026306e+06,256,0.3,0.3,64,0.2,0.4,96,0.001639
8,08,1.026346e+06,96,0.2,0.2,256,0.4,0.2,128,0.004951
9,09,1.026284e+06,256,0.0,0.1,256,0.0,0.3,80,0.000135


In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first LSTM layer is {best_hps.get('units1')},
the optimal dropout rates are {best_hps.get('dropout1')} for the first dropout layer and {best_hps.get('dropout2')} for the second dropout layer,
the optimal number of units in the second LSTM layer is {best_hps.get('units2')}, and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first LSTM layer is 160,
the optimal dropout rates are 0.2 for the first dropout layer and 0.0 for the second dropout layer,
the optimal number of units in the second LSTM layer is 160, and the optimal learning rate for the optimizer
is 0.00010301017394224362.



In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

Epoch 1/50
46/46 [==============================] - 7s 37ms/step - loss: 1013168.9375 - val_loss: 1026282.5625
Epoch 2/50
46/46 [==============================] - 1s 21ms/step - loss: 1013168.3125 - val_loss: 1026285.1875
Epoch 3/50
46/46 [==============================] - 1s 16ms/step - loss: 1013167.7500 - val_loss: 1026287.8125
Epoch 4/50
46/46 [==============================] - 1s 12ms/step - loss: 1013166.6875 - val_loss: 1026290.4375
Epoch 5/50
46/46 [==============================] - 1s 12ms/step - loss: 1013166.4375 - val_loss: 1026291.0000
Epoch 6/50
46/46 [==============================] - 1s 12ms/step - loss: 1013165.5625 - val_loss: 1026295.1250
Epoch 7/50
46/46 [==============================] - 1s 12ms/step - loss: 1013164.7500 - val_loss: 1026296.9375
Epoch 8/50
46/46 [==============================] - 1s 12ms/step - loss: 1013164.3125 - val_loss: 1026301.3750
Epoch 9/50
46/46 [==============================] - 1s 12ms/step - loss: 1013162.8750 - val_loss: 1026306.3125
E

In [ ]:
predicted_prices = model.predict(X_test_scaled)

12/12 [==============================] - 1s 3ms/step


In [ ]:
print("Test MSE:", np.mean((predicted_prices - y_test)**2))

Test MSE: 428645.1446262591


### Plot the comparison between actual and predicted value

In [ ]:
# y_close_test <- use value before minmax scaling
y_close_test = df_test_plot['close']
LSTM_pred = predicted_closing_prices.copy()

In [ ]:
import matplotlib.pyplot as plt

date_val = pd.to_datetime(y_close_test.index)
LSTM_close = y_close_test + (LSTM_pred - y_test)
fig = plt.figure(figsize=(13,8))
plt.plot(y_close_test, color='blue', linewidth=2, label='Actual')
plt.plot(LSTM_close, color='pink', linestyle='dashed',
linewidth=2, label="LSTM")
plt.title('Bitcoin Price Prediction', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Price', fontsize=15)
plt.legend()

In [ ]:
# Model architecture
tf.keras.utils.set_random_seed(4002)

model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, validation_split=0.2, verbose=1)

In [ ]:
# Predict
predictions = model.predict(X_test_scaled)

In [ ]:
# Evaluate
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print(f'Test MSE: {mse}')